In [13]:
import pandas as pd
import re
import kss
from tqdm import tqdm
from koalanlp.proc import SentenceSplitter
from koalanlp import API
from koalanlp.Util import initialize, finalize

initialize(HNN="LATEST")

splitter = SentenceSplitter(API.HNN)

tqdm.pandas()

[koalanlp.jip] [INFO] Latest version of kr.bydelta:koalanlp-hnn (2.1.4) will be used.
[root] Java gateway started with port number 34613
[root] Callback server will use port number 25334
[koalanlp.jip] JVM initialization procedure is completed.


In [16]:
file_path = "../database/review_2_naver.csv"
data = pd.read_csv(file_path, encoding="utf-8")

In [17]:
data[:5]

,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028
1,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출
2,609182b3f5c0adc042e03b0a,5,잘 머물다 갑니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12752105,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,5,그때그시절처렁
3,609182b3f5c0adc042e03b0b,5,최근에 침대가 너무 삐걱거림을 느끼고 있네요,2,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12751866,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,4,동별
4,609182b3f5c0adc042e03b13,5,잘 쉬다 가요,4,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,2,5,12747672,[특가야놀자☆10일간 Spring Flash Sale] Premier King,3,폭신폭신꿀보0006


In [18]:
print(data.shape)

(98998, 15)


In [19]:
def clean_text(x):
    x = re.sub('\.{2,}', '\.', x)
    x = re.sub('\!{2,}', '\!', x)
    x = re.sub(r'\+', '', x)
    x = re.sub(r'\-', '', x)
    x = re.sub('\n', '\.', x)
    x = re.sub('\s+', ' ', x)
    x = re.sub('[\ㄱ-ㅎ\ㅏ-ㅣ]', '', x)
    x = re.sub('[-=+,#/\?:^$@*\'※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', x)
    x = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z\.\!\?\,\%\& ]', '', x)

    return x

In [20]:
data["sentence"] = data["content"].progress_apply(lambda x: clean_text(str(x)))
data[0:5]

100%|██████████| 98998/98998 [00:01<00:00, 87913.54it/s]


,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user,sentence
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028,영등포에서는 이곳만 와요 너무 좋습니다
1,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다
2,609182b3f5c0adc042e03b0a,5,잘 머물다 갑니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12752105,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,5,그때그시절처렁,잘 머물다 갑니다
3,609182b3f5c0adc042e03b0b,5,최근에 침대가 너무 삐걱거림을 느끼고 있네요,2,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12751866,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,4,동별,최근에 침대가 너무 삐걱거림을 느끼고 있네요
4,609182b3f5c0adc042e03b13,5,잘 쉬다 가요,4,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,2,5,12747672,[특가야놀자☆10일간 Spring Flash Sale] Premier King,3,폭신폭신꿀보0006,잘 쉬다 가요


In [21]:
# def sentence_split(df, col, file_name):
def sentence_split(df, col):
    """
    split Korean sentences with kss
    """

    sentence = df[col]
    sent = []

    for i in tqdm(range(len(sentence))):
        sent.append(kss.split_sentences(sentence[i]))

    sent2 = []
    for i in tqdm(sent):
        sent2.append(splitter(i))
    sent2 = pd.Series(sent2)

    df["fixed"] = sent2
    df = df.explode("fixed")

    return df

In [22]:
# sentence_split(data, col="content", file_name="../database/review_1.csv")
data = sentence_split(data, col="sentence")
data[:5]

100%|██████████| 98998/98998 [00:37<00:00, 2614.73it/s]


,_id,cleanliness,content,convenience,date,hotelAddr,hotelGrade,hotelLoc,hotelName,kindness,position,reviewID,room,totalScore,user,sentence,fixed
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028,영등포에서는 이곳만 와요 너무 좋습니다,영등포에서는 이곳만 와요
0,609182b3f5c0adc042e03afd,5,영등포에서는 이곳만 와요 너무 좋습니다,5,2021-05-04,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12759201,[특가야놀자☆선착순 고층 룸UP+25시간] Premier King,5,지금사랑해0028,영등포에서는 이곳만 와요 너무 좋습니다,너무 좋습니다
1,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,체크인 시간이 너무너무 깁니다
1,609182b3f5c0adc042e03b08,5,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,1,5,12752310,[특가야놀자☆선착순 고층 룸UP+27시간] Premier King,3,엄마구출,체크인 시간이 너무너무 깁니다 빨리할 수 있는 방법을 구상해야 할 것 같습니다,빨리할 수 있는 방법을 구상해야 할 것 같습니다
2,609182b3f5c0adc042e03b0a,5,잘 머물다 갑니다,5,2021-05-03,서울특별시 영등포구 영등포동1가 143-3,4성급,영등포역 5번출구 도보 3분,페어필드 바이 메리어트 서울,5,5,12752105,[특가야놀자☆선착순 고층 룸UP+27시간스테이] Premier Twin,5,그때그시절처렁,잘 머물다 갑니다,잘 머물다 갑니다


In [24]:
data.to_csv("../database/review_3_naver.csv", encoding="utf-8", index=False)

In [25]:
finalize()

True